In [ ]:
import sys

iteration = sys.argv[1]


exit()
import random
from tensorflow import keras
from keras.layers import Input, Dense
from keras.models import Model
from keras import backend as K

import math as math
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import plot_model
from keras.constraints import UnitNorm
from keras.layers import Lambda
from keras.losses import binary_crossentropy

ndim=3
#############
# functions #
#############
#specify function to sample from latent dim
def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim),
                              mean=0., stddev=1)
    return z_mean + K.exp(z_log_sigma) * epsilon

# cov2cor

def cov2cor(x):
    sds=np.diag(1/np.sqrt(np.diag(x)))
    out=np.matmul(np.matmul(sds,x),sds)
    return out

#function to ensure simple structure
def a_reg2(a):
    b = a * structure_matrix
    return b

def inv_factors(a_est, a_true, theta_est=None):
    """
    Helper function that inverts factors when discrimination values are mostly negative this improves the
    interpretability of the solution
        theta: NxP matrix of theta estimates
        a: IxP matrix of a estimates

        returns: tuple of inverted theta and a paramters
    """
    for dim in range(a_est.shape[1]):
        if pearsonr(a_est[:,dim], a_true[:,dim])[0] < 0:
            a_est[:, dim] *= -1
            theta_est[:, dim] *=-1

    return a_est, theta_est

#function to introduce covariances
force_chol=K.cast(np.full((ndim,ndim),1),"float32")
force_chol1=K.cast(np.tril(force_chol,k=-1),"float32")
force_chol2=K.cast(np.diag([1]*ndim),"float32")

def c_reg(c):
    d= c*force_chol1
    e=d+force_chol2
    return e;   

##############
# Data generation
###############
N=1000                  # number of subjects
nit=60                  # number of items 
ndim=3                  #number of dimensions
nit_dim=int(nit/ndim)   #number of items per dimension
max_nepoch=50000        #max number of iterations

X=np.full((N,nit),999)      # empty matrix for item scores
prob=np.full((N,nit),0.99)   # empty matrix for probability correct
a=np.full((nit,ndim),0.0)      # empty matrix for discrimination parameters 

covMat=np.full((ndim,ndim),.3)  # covariance matrix of dimensions
np.fill_diagonal(covMat,1)
theta=np.random.multivariate_normal([0]*ndim,covMat,N)   # draw values for the dimensions

structure_matrix=np.full((nit,ndim),0.0)

for i in range(0,ndim):
    ax=np.random.uniform(.5,1.5,nit_dim)            #draw discrimination parameters from uniform distribution
    a[range(nit_dim*i, nit_dim*(i+1)),i] = ax
    structure_matrix[range(nit_dim*i, nit_dim*(i+1)),i]=1
structure_matrix = K.cast(K.transpose(structure_matrix),"float32")   # simple structure configuration

    
b=np.tile(np.linspace(-3,3,nit_dim,endpoint=True),ndim)   # decoder intercepts

for i in range(0,nit):
    for p in range(0,N):
        prob[p,i]=1/(1+math.exp(-(sum(a[i,:]*theta[p,:])+b[i])))       # probability correct
        X[p,i]=np.random.binomial(1,prob[p,i])                          # draw item scores on basis of prob correct

##############
# Set up VAE
###############
encoding_dim=np.ceil((nit+ndim)/2)
latent_dim=ndim  #dimensions in VAE
batch_size=N  # sample size 


x=Input(batch_shape=(batch_size, nit))    # input layer
e1 = Dense(encoding_dim, activation='elu')(x)
z_mean = Dense(latent_dim,activation='linear')(e1)                   # specify person mean latent dim
z_log_sigma = Dense(latent_dim,activation='linear')(e1)              # specify person sd latent dim



#sample from latent dim
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])

#encoder: connect the input to the latent dim parameters
encoder = Model(x, [z_mean, z_log_sigma, z])
       

# build decoder model
latent_inputs = Input(shape=(latent_dim,))  # input for the decoder model: the latent dimensions
#outputs1 = Dense(latent_dim, activation='linear', kernel_constraint=c_reg,use_bias=False)(latent_inputs)  # the "cholesky layer"
outputs2 = Dense(nit, activation='sigmoid', kernel_constraint=a_reg2)(latent_inputs)  # connect input to output
 
 
# instantiate decoder model
decoder = Model(latent_inputs, outputs2)  # specify the decode model

# instantiate VAE model
outputs = decoder(encoder(x)[2])    # full output encoder -> decoder -> output
vae = Model(x, outputs)


# specify the loss function to estimate the model
reconstruction_loss = binary_crossentropy(x,outputs)   

reconstruction_loss *= nit
kl_loss = 1 + 2*z_log_sigma - K.square(z_mean) - K.exp(2*z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)

vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
 
# fit the model
callback = keras.callbacks.EarlyStopping(monitor='loss', min_delta=1e-8,patience=500,verbose=1,mode="min")

hist=vae.fit(X, 
        epochs=100,
        batch_size=batch_size,verbose=0)

hist=vae.fit(X, 
        epochs=max_nepoch,
        batch_size=batch_size, verbose=0, callbacks=[callback])

loss=hist.history['loss']

pars=vae.get_weights()

theta_est = encoder(X)

##############
# Check correlations
###############

# this *should* give the correct correlations (all .3):
L=pars[-3]
print(cov2cor(np.matmul(L,np.transpose(L))))

#doesnt look like it. BUT:

theta_est = np.matmul(theta_est[0],L)
for i in range(3):
    if np.corrcoef(theta_est[:,i], theta[:,i]) < 0:
        theta_est *= -1

np.corrcoef(np.transpose(theta_est))

# using the biased lambdra matrix (L) and the estimates of mu
# from the latent variables (g), the actual latent variables (f) seem to have the correct correlations 

# dit krijg ik als correlaties (die negatieve cors zijn niet erg, komt doordat schaal is omgeklapt, 
# dat kan ik ook zien aan de item ladingen, die zijn negatief voor factor 1 en 3:
 #   array([[ 1.        , -0.30513961,  0.39002258],
 #          [-0.30513961,  1.        , -0.32839606],
 #          [ 0.39002258, -0.32839606,  1.        ]])
 
 # en uit R (lavaan):
 # f1 ~~                                               
 # f2                0.290    0.038    7.670    0.000
 # f3                0.374    0.038    9.968    0.000
#  f2 ~~                                               
#    f3                0.317    0.039    8.196    0.000
   
#lijkt heel aardig overeen te komen, mÃ¡Ã¡r het zijn geen schattingen, de schattingen zelf (in L) zijn biased.
#we moeten even nadenken hoe we dat willen oplossen (meerdere opties) 
     
#heel soms worden de off diagnal elements in L groter dan 1. Dit zou geen probleem moeten zijn, maar dan is de boel flinkt
# biased, bij mij kwam het 1x voor en toen waren de correlaties tussen de mu schattingen echt hoog. Hopelijk komt
# dat niet vaak voor en anders moeten we misschien zorgen dat die off-diagonal elements tussen de -1 en 1 blijven.
cor = cov2cor(np.matmul(L,np.transpose(L)))

est = np.array([cor[1,0], cor[2,0], cor[1,2]])
true = np.array([covMat[1,0], covMat[2,0], covMat[1,2]])

mse = np.mean((est-true)**2)

f = open(f"test_{iteration}.txt", "w+")
f.write(mse)

In [7]:
import numpy as np
import torch.nn.functional as F
from keras.losses import binary_crossentropy
import tensorflow.keras.backend as K
import torch

true = np.array([1,1])
pred = np.array([.4, .6])

print(F.binary_cross_entropy(torch.Tensor(pred), torch.Tensor(true)))
print(binary_crossentropy(K.constant(true), K.constant(pred)))

tensor(0.7136)
tf.Tensor(0.71355796, shape=(), dtype=float32)


In [3]:
pars

NameError: name 'pars' is not defined

In [ ]:
theta_est[0].shape

In [5]:
import sys

iteration = sys.argv[1]


exit()
import random
from tensorflow import keras
from keras.layers import Input, Dense
from keras.models import Model
from keras import backend as K

import math as math
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import plot_model
from keras.constraints import UnitNorm
from keras.layers import Lambda
from keras.losses import binary_crossentropy

ndim=3
#############
# functions #
#############
#specify function to sample from latent dim
def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim),
                              mean=0., stddev=1)
    return z_mean + K.exp(z_log_sigma) * epsilon

# cov2cor

def cov2cor(x):
    sds=np.diag(1/np.sqrt(np.diag(x)))
    out=np.matmul(np.matmul(sds,x),sds)
    return out

#function to ensure simple structure
def a_reg2(a):
    b = a * structure_matrix
    return b

def inv_factors(a_est, a_true, theta_est=None):
    """
    Helper function that inverts factors when discrimination values are mostly negative this improves the
    interpretability of the solution
        theta: NxP matrix of theta estimates
        a: IxP matrix of a estimates

        returns: tuple of inverted theta and a paramters
    """
    for dim in range(a_est.shape[1]):
        if pearsonr(a_est[:,dim], a_true[:,dim])[0] < 0:
            a_est[:, dim] *= -1
            theta_est[:, dim] *=-1

    return a_est, theta_est

#function to introduce covariances
force_chol=K.cast(np.full((ndim,ndim),1),"float32")
force_chol1=K.cast(np.tril(force_chol,k=-1),"float32")
force_chol2=K.cast(np.diag([1]*ndim),"float32")

def c_reg(c):
    d= c*force_chol1
    e=d+force_chol2
    return e;   

##############
# Data generation
###############
N=1000                  # number of subjects
nit=60                  # number of items 
ndim=3                  #number of dimensions
nit_dim=int(nit/ndim)   #number of items per dimension
max_nepoch=50000        #max number of iterations

X=np.full((N,nit),999)      # empty matrix for item scores
prob=np.full((N,nit),0.99)   # empty matrix for probability correct
a=np.full((nit,ndim),0.0)      # empty matrix for discrimination parameters 

covMat=np.full((ndim,ndim),.3)  # covariance matrix of dimensions
np.fill_diagonal(covMat,1)
theta=np.random.multivariate_normal([0]*ndim,covMat,N)   # draw values for the dimensions

structure_matrix=np.full((nit,ndim),0.0)

for i in range(0,ndim):
    ax=np.random.uniform(.5,1.5,nit_dim)            #draw discrimination parameters from uniform distribution
    a[range(nit_dim*i, nit_dim*(i+1)),i] = ax
    structure_matrix[range(nit_dim*i, nit_dim*(i+1)),i]=1
structure_matrix = K.cast(K.transpose(structure_matrix),"float32")   # simple structure configuration

    
b=np.tile(np.linspace(-3,3,nit_dim,endpoint=True),ndim)   # decoder intercepts

for i in range(0,nit):
    for p in range(0,N):
        prob[p,i]=1/(1+math.exp(-(sum(a[i,:]*theta[p,:])+b[i])))       # probability correct
        X[p,i]=np.random.binomial(1,prob[p,i])                          # draw item scores on basis of prob correct

##############
# Set up VAE
###############
encoding_dim=np.ceil((nit+ndim)/2)
latent_dim=ndim  #dimensions in VAE
batch_size=N  # sample size 


x=Input(batch_shape=(batch_size, nit))    # input layer
e1 = Dense(encoding_dim, activation='elu')(x)
z_mean = Dense(latent_dim,activation='linear')(e1)                   # specify person mean latent dim
z_log_sigma = Dense(latent_dim,activation='linear')(e1)              # specify person sd latent dim



#sample from latent dim
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])

#encoder: connect the input to the latent dim parameters
encoder = Model(x, [z_mean, z_log_sigma, z])
       

# build decoder model
latent_inputs = Input(shape=(latent_dim,))  # input for the decoder model: the latent dimensions
#outputs1 = Dense(latent_dim, activation='linear', kernel_constraint=c_reg,use_bias=False)(latent_inputs)  # the "cholesky layer"
outputs2 = Dense(nit, activation='sigmoid', kernel_constraint=a_reg2)(latent_inputs)  # connect input to output
 
 
# instantiate decoder model
decoder = Model(latent_inputs, outputs2)  # specify the decode model

# instantiate VAE model
outputs = decoder(encoder(x)[2])    # full output encoder -> decoder -> output
vae = Model(x, outputs)


# specify the loss function to estimate the model
reconstruction_loss = binary_crossentropy(x,outputs)   

reconstruction_loss *= nit
kl_loss = 1 + 2*z_log_sigma - K.square(z_mean) - K.exp(2*z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)

vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
 
# fit the model
callback = keras.callbacks.EarlyStopping(monitor='loss', min_delta=1e-8,patience=500,verbose=1,mode="min")
